# Georeferencing

This code uses the geographical information stored in the original unprocessed orthomosaic and applies it to a copy of the Seagrass binary tif, which is the end result of the previous section. The output is a binary tif that can be dragged and dropped into QGIS and has the correct geographical and projection information.

*To do:*
- Polygonize/vectorize the seagrass and save it as a shapefile
- Calculate basic statistics: Seagrass Area in m2, % of total area, average patch size & std maybe?

In [1]:
# import libraries
from osgeo import gdal

In [2]:
image_name = "AK_tryout"
src_path = rf"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\{image_name}.tif"
pro_path = rf"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\{image_name}_SeagrassBinary.tif"
dst_path = rf"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\{image_name}_GeoRef.tif"

In [9]:
# open original and processed tif in gdal
src_ds = gdal.Open(src_path)
pro_ds = gdal.Open(pro_path)

# get geotransform data from the original tif and safe as variable
geo_data = src_ds.GetGeoTransform()

# get driver
fileformat = "GTiff"
driver = gdal.GetDriverByName(fileformat)

# create a copy of the processed image in the correct format (GTiff), then apply geotransform information to the copy
dst_ds = driver.CreateCopy(dst_path, pro_ds, strict=0)
dst_ds.SetGeoTransform(geo_data)

# assign the right projection
src_projection = src_ds.GetProjection()
dst_ds.SetProjection(src_projection)

if geo_data == dst_ds.GetGeoTransform():
    print(f"The binary file has been successfully georeferenced. The pixelsize is {geo_data[1]} metres")

# close all datasets
dst_ds = None
src_ds = None
pro_ds = None

The binary file has been successfully georeferenced. The pixelsize is 0.05025239034331673 metres


## Part 2: Basic Statistics

In [4]:
# define the function that computes the seagrass area based on the size of one pixel and the number of white pixels

def calculate_white_area(image, pixel_size):
 
    # Check if the image was loaded correctly
    if image is None:
        print("Error: Image not loaded correctly.")
        return

    # Count the number of white pixels (value 255)
    white_pixel_count = cv2.countNonZero(image)

    # Each white pixel represents an area on the ground based on the GSD
    area_per_pixel_in_cm2 = pixel_size ** 2

    # Total area in square centimeters
    area_cm2 = white_pixel_count * area_per_pixel_in_cm2

    # Convert area to square meters
    area_m2 = area_cm2 / 10000

    # Convert area to hectares
    area_hectares = area_m2 / 10000

    # Print the results
    print(f"Number of white pixels: {white_pixel_count}")
    print(f"Area covered by one pixels: {pixel_size*pixel_size} square centimeters")
    print(f"Covered area: {round(area_cm2, 0)} square centimeters")
    print(f"Covered area: {round(area_m2, 2)} square meters")
    print(f"Covered area: {round(area_hectares, 4)} hectares")

In [6]:
# Define inputs and run function

# Pixel size in cm, from GDAL GeoTransform
pixel_size = geo_data[0]

# Calculate and display the white area
calculate_white_area(image_with_filled_patches, pixel_size)

NameError: name 'image_with_filled_patches' is not defined

In [119]:
dst_ds.polygonize(1, 1)


AttributeError: 'Dataset' object has no attribute 'polygonize'

In [120]:
dst_ds = None

## More tryouts

In [5]:
import cv2
import numpy as np

In [9]:
imo = cv2.imread(r"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\AK_tryout.tif")
imp = cv2.imread(r"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\Seagrass_binary_AK_tryout.tif")

In [12]:
np.shape(imo)

(3568, 6217, 3)

In [13]:
# check if the processed image still has the same shape as the original one
# this is necessary because geotiffs use the point of origin (in the top left corner) as a geographical reference 
# if the image does not have the same shape anymore, the point of origin will not be the same either
# if this is the case, the processed image has to be manually georeferenced in QGIS, see the tutorial xxx
if np.shape(imo[:,:,0]) == np.shape(imp[:,:,0]):
    print("Proceed with the following step.")
else:
    print("The shape of the image was changed during processing, meaning the point of origin can't be assigned from the original to the processed image anymore. Instead, the image has to be georeferenced manually in QGIS.")

Proceed with the following step.


to do: 
- set if-clause to check for loaded pictures
- set if-clause to check same size
- set projection to same as original pictures
- figure out how to make into vector / pretty symbology

## With GDAL

In [64]:
# define picture paths
src_filename = r"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\AgiosKirikos-Bay1-15022025.tif"
dst_filename = r"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\AgiosKirikos-Bay1-15022025_GeoRef.tif"
pro_filename = r"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\Seagrass_binary_AgiosKirikos-Bay1-15022025.tif"

In [65]:
# open original and processed tif in gdal
src_ds = gdal.Open(src_filename)
pro_ds = gdal.Open(pro_filename)

# get geotransform data from the original tif and safe as variable
geo_data = src_ds.GetGeoTransform()

# get driver
fileformat = "GTiff"
driver = gdal.GetDriverByName(fileformat)

# create a copy of the processed image in the correct format (GTiff), then apply geotransform information to the copy
dst_ds = driver.CreateCopy(dst_filename, pro_ds, strict=0)
dst_ds.SetGeoTransform(geo_data)
dst_ds.GetGeoTransform()

# assign the right projection
src_projection = src_ds.GetProjection()
dst_ds.SetProjection(src_projection)

# close all files
dst_ds = None
src_ds = None
pro_ds = None

# Tryouts

## Assign Projection

In [4]:
src_ds = gdal.Open(src_filename)
dst_ds = gdal.Open(dst_filename)

C:\Users\aline\anaconda3\envs\GeoReference\Lib\site-packages\osgeo\gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [5]:
src_projection = src_ds.GetProjection()
src_projection

'PROJCS["WGS 84 / UTM zone 35N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",27],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32635"]]'

In [6]:
dst_ds.RasterCount

1

In [7]:
dst_ds.SetProjection(src_projection)

0

In [8]:
dst_ds = None
src_ds = None

## figuring out filenames

In [ ]:
# import libraries
from osgeo import gdal

In [ ]:
# define picture paths 
#original tif for the georeferencing, processed tif for the input data, destination tif is the name of the newly created one
src_filename = r"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\image.tif"
pro_filename = r"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\Seagrass_binary_image.tif"
dst_filename = r"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\Lipsi2_GeoRef.tif"

source_path = r"C:\Users\aline\Seagrass Mapping Drone\Tifffiles"
image_name = "Lemnos-Site11.tif"
image = cv2.imread(os.path.join(source_path, image_name))
results_path = r"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs"

image_name
rf"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\{image_name}"

In [15]:
pro_path

'C:\\Users\\aline\\Seagrass Mapping Drone\\Processed tiffs\\Seagrass_binary_image.tif'

In [16]:
pic = gdal.Open(pro_path)

In [17]:
pic.GetProjection()

''

In [57]:
image_name = "AgiosKirikos-Bay1-15022025"
src_path = rf"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\{image_name}"
pro_path = rf"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\Seagrass_binary_{image_name}"
dst_path = rf"C:\Users\aline\Seagrass Mapping Drone\Processed tiffs\GeoRef_{image_name}"

In [58]:
# open original and processed tif in gdal
src_ds = gdal.Open(src_path)
pro_ds = gdal.Open(pro_path)

# get geotransform data from the original tif and safe as variable
geo_data = src_ds.GetGeoTransform()

# get driver
fileformat = "GTiff"
driver = gdal.GetDriverByName(fileformat)

# create a copy of the processed image in the correct format (GTiff), then apply geotransform information to the copy
dst_ds = driver.CreateCopy(dst_path, pro_ds, strict=0)
dst_ds.SetGeoTransform(geo_data)
dst_ds.GetGeoTransform()

# apply the same projection to the georeferenced tif
src_projection = src_ds.GetProjection()
dst_ds.SetProjection(src_projection)

# close all files
dst_ds = None
src_ds = None
pro_ds = None

AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'

In [24]:
src_ds = gdal.Open(src_path)

In [25]:
geo_data = src_ds.GetGeoTransform()

AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'

In [28]:
src_path = r"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\image.tif"
src_ds = gdal.Open(src_filename)

In [29]:
geo_data = src_ds.GetGeoTransform()

In [41]:
image_name = "AgiosKirikos-Bay1-15022025"
src_path = rf"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\{image_name}"

In [46]:
src_path = r"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\AgiosKirikos-Bay1-15022025"

In [50]:
src_path

'C:\\Users\\aline\\Seagrass Mapping Drone\\Tifffiles\\AgiosKirikos-Bay1-15022025'

In [51]:
src_ds = None

In [52]:
src_ds = gdal.Open(src_path)

In [53]:
gt = src_ds.GetGeoTransform()

AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'

In [39]:
src_ds = None

In [36]:
pro_ds = None
dst_ds = None

In [40]:
image_name = "AgiosKirikos-Bay1-15022025"
src_path = rf"C:\Users\aline\Seagrass Mapping Drone\Tifffiles\{image_name}"

src_ds = gdal.Open(src_path)

geo_data = src_ds.GetGeoTransform()

AttributeError: 'NoneType' object has no attribute 'GetGeoTransform'